# Pound Drever Hall Analysis

This Notebook will contain design specs, analysis, calculations etc for the Pound Drever Hall Project

## Purpose of the Project
An optical cavity is formed by two or more mirrors which highly reflect incident light except for a set of narrow <i>resonant</i> frequencies, for which the cavity is highly transmissive. 

The point of this project is to stabilize the length of a cavity so that an incoming "lock beam" will always be resonant to it (i.e. experience maximum transmission). 

The Pound Drever Hall (PDH) technique is a tried, tested and true method for accomplishing this. Essentially PDH works by modulating the incoming light creating sidebands at frequecy $\pm \Omega$ that are reflected from the cavity, and using these sidebands to create an error signal proportional to the deviation of the cavity from resonance.

This project aims to create the electronics, mechanical assemble, and optics to create a PDH lock


### TODO ###
- Tutorial of Physics
- Calculation of parameters
- Add references

## DESIGN SPECS
(Calculations to follow)

### What modulation frequency do we want?
- Our cavity will have a length of around 2.5 to 10 cm. This corresponds to an FSR of 1.5 to 6 GHz. Our relflectivity is on the order of R ~ 1 - 0.001. This corresponds to a finesse of about 1000*pi, let's just call it 3000. Given this our linewidth (FWHM) is between 1 and 4 MHz. Since we want our sidebands to be about 10x a linewidth, we really expect our primary frequency to be about 10 to 40 MHz.

What are the relevant signal
- It can be shown that the optimal slope occurs when each sideband has half the power of the carrier. We will actually need fairly high bandwidth detectors (100 MHz say). Our light will have a power in the order of 100 &mu;W and so we will have each sideband at about 25 &mu;W. However, we may not get down to this level so that we should expect a signal as low as 5&mu;W at the modulation frequency. With a transimpedance gain of 100k&Omega; and a responsivity of 0.5 A/W we can thus expect a signal level of about 250 mV. This seems high ... however if this is not achievable with a single stage, we can have a second gain stage at the output. 

What kind of load are we driving?
- We will drive a Piezo connected to a mirror (which is essentially a capacitive load, w/ $C_{DC} \approx 1~\mu F$
- We will need a fairly large voltage gain here, since teh piezo will require about 100 V to move a full stroke. 
- Our mechanical bandwidth is about 10 kHz, so we don't need anything faster than that.